# I created the OpenNMT Pytorch tutorial using Colab.

***First Go to Runtime and  change the runtime type to GPU.***


<br>
 Copyright Park Chanjun
<br>
 Email: bcj1210@naver.com










# Git Clone
First Git clone the OpenNMT source

In [ ]:
!git clone https://github.com/OpenNMT/OpenNMT-py

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 17656, done.
remote: Counting objects: 100% (615/615), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 17656 (delta 365), reused 489 (delta 289), pack-reused 17041
Receiving objects: 100% (17656/17656), 273.79 MiB | 27.99 MiB/s, done.
Resolving deltas: 100% (12673/12673), done.


# Please install requirements.txt use by pip

> Error : You must restart the runtime in order to use newly installed versions.<br>
Solution : Click Restart Runtime => Redo



In [ ]:
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 5.4 MB/s 
     |████████████████████████████████| 73 kB 1.5 MB/s 
     |████████████████████████████████| 16.3 MB 43.0 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 1.2 MB 44.7 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


# Theory explanation

**Machine translation is a field of natural language processing, meaning that computers translate one language into another.**

Rule based, and statistical based, and recently we are using Deep Learning-based machine translation.

Learn how to build a real machine translation system and how the system pipeline is structured. Most of these courses can be applied to basic natural language processing problems as well as machine translation.

**Step**



**1.   Data Collection**

Parallel corpus is collected from various sources. It is possible to collect news texts, drama / movie subtitles, Wikipedia, etc., as well as data sets for evaluation of translation systems disclosed by WMT, a machine translation competition, and use them in translation systems.


**2.   Cleaning**

The collected data must be refined. The refinement process includes sorting sentences by corpus in both languages, and eliminating noise such as special characters.


**3. Subword Tokenization**

Refine spacing using the POS tagger or segmenter for each language. English may have refinement issues in upper / lower case.
After the spacing is refined, use Byte Pair Encoding (BPE) using public tools such as Subword or WordPiece. This allows you to perform additional segments and construct a vocabulary list. At this time, the segmented models learned for the BPE segment should be kept for future use.


**4. Train**

Train the seq2seq model using prepared datasets. Depending on the amount, you can train with a single GPU, or use multiple GPUs in parallel to reduce training time.


**5. Translate**

Now that the model has been created, you can start translating.


**6. Detokenization**

Even after the translation process is finished, it is still in a segment, so it is different from the actual sentence structure used by real people. Thus, when you perform a detoxification process, it is returned in the form of the actual sentence.


**7. Evaluating**

Quantitative evaluation is performed on the sentence thus obtained. BLEU is a quantitative evaluation method for machine translation. You can see which model is superior by comparing it to the BLEU score you are comparing.

# Tutotorial Start


Assume that you have data collection and refinement and start the tutorial.

Use the data provided by OpenNMT-py.

Locate in **OpenNMT-py/data**



# Subword Tokenization

We use Byte Pair Encoding for Subword Tokenization

https://www.aclweb.org/anthology/P16-1162

i => input<br>
o ==> Output(*.code)<br>
s ==> Symbol<br>

learn_bpe ==> make code<br>
apply_bpe ==> apply subwordTokenization<br>

src-train, src-val,test ==> Need to apply src.code<br>
tgt-train,tgt-val ==> Need to apply tgt.code

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i OpenNMT-py/data/src-train.txt -o OpenNMT-py/data/src.code -s 10000

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i OpenNMT-py/data/tgt-train.txt -o OpenNMT-py/data/tgt.code -s 10000

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/src.code -i OpenNMT-py/data/src-train.txt -o OpenNMT-py/data/src-train-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/src.code -i OpenNMT-py/data/src-val.txt -o OpenNMT-py/data/src-val-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/src.code -i OpenNMT-py/data/src-test.txt -o OpenNMT-py/data/src-test-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/tgt.code -i OpenNMT-py/data/tgt-train.txt -o OpenNMT-py/data/tgt-train-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/tgt.code -i OpenNMT-py/data/tgt-val.txt -o OpenNMT-py/data/tgt-val-bpe.txt

# **Preprocess the data**

We will be working with some example data in data/ folder.

The data consists of parallel source (src) and target (tgt) data containing one sentence per line with tokens separated by a space:

1. src-train.txt

2. tgt-train.txt

3. src-val.txt

4. tgt-val.txt


Train data and validataion data are required for machine translation training.

Validation files are required and used to evaluate the convergence of the training. It usually contains no more than 5000 sentences.


> If you think about it briefly, you can specify the path of train data and validation data, and specify the path and name to save in -save_data.

> If you want to set vocab size add below command
<br>
-src_vocab_size 32000 -tgt_vocab_size 32000

The vocab size is usually 32000.

In [ ]:
!python OpenNMT-py/build_vocab.py -config OpenNMT-py/demo-src-tgt.yml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-01-25 15:25:14,821 INFO] Counter vocab from 10000 samples.
[2022-01-25 15:25:14,821 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-01-25 15:25:14,836 INFO] corpus_1's transforms: TransformPipe()
[2022-01-25 15:25:14,837 WARNING] Empty line exists in corpus_1#4.
[2022-01-25 15:25:15,243 INFO] Counters src:9911
[2022-01-25 15:25:15,243 INFO] Counters tgt:10152


# **Train the data(Basic)**

This is simple Train command use 2-layer LSTM with 500 hidden units on both the encoder/decoder.

If you want to use GPU , try add  below command (example use 2 GPU)
>-world_size 2 -gpu_ranks 0 1

Let's Check Available GPU


In [ ]:
!nvidia-smi

Tue Jan 25 15:24:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python OpenNMT-py/train.py -config OpenNMT-py/demo-src-tgt.yml

[2022-01-25 15:25:24,578 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-01-25 15:25:24,578 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-01-25 15:25:24,578 INFO] Missing transforms field for valid data, set to default: [].
[2022-01-25 15:25:24,578 INFO] Parsed 2 corpora from -data.
[2022-01-25 15:25:24,578 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-01-25 15:25:24,579 INFO] Loading vocab from text file...
[2022-01-25 15:25:24,579 INFO] Loading src vocabulary from OpenNMT-py/data/demo.vocab.src
[2022-01-25 15:25:24,598 INFO] Loaded src vocab has 9911 tokens.
[2022-01-25 15:25:24,603 INFO] Loading tgt vocabulary from OpenNMT-py/data/demo.vocab.tgt
[2022-01-25 15:25:24,625 INFO] Loaded tgt vocab has 10152 tokens.
[2022-01-25 15:25:24,631 INFO] Building fields with vocab in counters...
[2022-01-25 15:25:24,644 INFO]  * tgt vocab size: 10156.
[2022-01-25 15:25:24,656 INFO]  * src vocab size

# **Train the data(Transformer)**

https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf


> If you get GPU-related errors, try halving batch_size

**Below is the full command, and if you want to know more about it, search about Transformer.**

!python OpenNMT-py/train.py -data OpenNMT-py/data/demo -save_model OpenNMT-py/data/model/model -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding -train_steps 200000 -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens -normalization tokens -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 -param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 -world_size 1 -gpu_rank 0  

In [ ]:
!python OpenNMT-py/train.py -config OpenNMT-py/demo-transformer.yml

[2022-01-25 15:27:03,648 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-01-25 15:27:03,648 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-01-25 15:27:03,648 INFO] Missing transforms field for valid data, set to default: [].
[2022-01-25 15:27:03,648 INFO] Parsed 2 corpora from -data.
[2022-01-25 15:27:03,648 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-01-25 15:27:03,649 INFO] Loading vocab from text file...
[2022-01-25 15:27:03,649 INFO] Loading src vocabulary from OpenNMT-py/data/demo.vocab.src
[2022-01-25 15:27:03,669 INFO] Loaded src vocab has 9911 tokens.
[2022-01-25 15:27:03,674 INFO] Loading tgt vocabulary from OpenNMT-py/data/demo.vocab.tgt
[2022-01-25 15:27:03,699 INFO] Loaded tgt vocab has 10152 tokens.
[2022-01-25 15:27:03,704 INFO] Building fields with vocab in counters...
[2022-01-25 15:27:03,718 INFO]  * tgt vocab size: 10156.
[2022-01-25 15:27:03,731 INFO]  * src vocab size

# **Translate**

Now that you have your model, you can start translating.

-model ==> Setting your model

Output predictions into pred.txt

In [ ]:
!python OpenNMT-py/translate.py -model OpenNMT-py/model/model_step_1000.pt -src OpenNMT-py/data/src-test.txt -output OpenNMT-py/data/pred.txt -replace_unk -verbose -beam_size 1

Streaming output truncated to the last 5000 lines.
PRED SCORE: -121.6331

[2022-01-25 15:59:40,271 INFO] 
SENT 870: ['An', 'inquiry', 'by', 'the', 'FDP', 'confirmed', 'this', '.']
PRED 870: Die Kommission hat die von der Rat .
PRED SCORE: -21.7536

[2022-01-25 15:59:44,663 INFO] 
SENT 871: ['Only', 'the', 'post', 'boxes', 'in', 'front', 'of', 'the', 'post', 'office', 'building', 'at', 'Ernst-Ludwig-Straße', '36', 'and', 'at', 'Jakob-Müller-Straße', '1', 'in', 'Hüttenfeld', 'are', 'still', 'emptied', '.']
PRED 871: Die Kommission hat die Frage der Lage in der Lage in der Lage in der Nähe der Nähe des Rates und in der Nähe .
PRED SCORE: -66.1419

[2022-01-25 15:59:44,663 INFO] 
SENT 872: ['Whether', 'there', 'are', 'still', 'Sunday', 'collections', 'in', 'Hofheim', 'and', 'Rosengarten', 'is', 'not', 'addressed', 'in', 'the', 'inquiry', 'by', 'the', 'FDP', '.']
PRED 872: Die Kommission ist nicht in der Fall , und die in der die von der Rat nicht mehr als auch nicht mehr als auch nicht .
P

# Detokenization

Even after the translation process is finished, it is still in a segment, so it is different from the actual sentence structure used by real people. Thus, when you perform a detoxification process, it is returned in the form of the actual sentence.

We Use "sed" for BPE Detokenization


In [ ]:
!sed -i 's/\@\@ //g' OpenNMT-py/data/pred.txt

# Evaluation Using BLEU

Quantitative evaluation is performed on the sentence thus obtained. BLEU is a quantitative evaluation method for machine translation. You can see which model is superior by comparing it to the BLEU score you are comparing.

https://www.aclweb.org/anthology/P02-1040

In [ ]:
# We don't have a reference file, but this command stays the same to get the bleu score for the translated sentences.

#!perl  OpenNMT-py/tools/multi-bleu-detok.perl OpenNMT-py/data/ref.txt < OpenNMT-py/data/pred.txt

ERROR: could not find reference file OpenNMT-py/data/ref.txt at OpenNMT-py/tools/multi-bleu-detok.perl line 40.


If you have Any Question Please Email to  "bcj1210@naver.com"